<a href="https://colab.research.google.com/github/AravindAryaPS/Arya/blob/master/DE_SOLUTION_Aravind_PS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1-- Environment setup and Data Loading.

In [ ]:
# Updating Installed packages in Google Colab Notebook
!apt-get update -y

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,155 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,104 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvid

In [ ]:
# For using Spark we need to setup JVM to run spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# downloading and unzip Apache Spark with Hadoop
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# Setup Environment for JAVA and Spark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# installing FindSpark library

!pip install -q findspark
import findspark
findspark.init()



In [ ]:
# import sparkSession from pyspark.sql and create a entry Point == Chess_Game

from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("Chess_Game").getOrCreate())

In [ ]:
# install gdown to read a file from Drive

!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import gdown to import the file folder to colab notebook

import gdown

# a folder
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

# same as the above, but with the folder ID
id = "1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(id=id, quiet=True, use_cookies=False)


['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

# #### Read all CSV in Gdrive Dictorery

In [ ]:


game_info = spark.read.load("/content/chess/chess_wc_history_game_info.csv",format='csv', sep=",",inferSchema="true",header=True)
moves = spark.read.load("/content/chess/chess_wc_history_moves.csv",format='csv', inferSchema="true", sep=",",header=True)
eco_codes = spark.read.load("/content/chess/eco_codes.csv",format='csv', sep=",",inferSchema="true",header=True)
print("All files loaded as DataFrame Sucessfully")

All files loaded as DataFrame Sucessfully


In [ ]:
# Upto above we downloaded the files from drive and readed by spark library

In [ ]:
#schema validation
game_info.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



In [ ]:
# viewing data
game_info.show(3)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
#data Validation
moves.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- move_no: integer (nullable = true)
 |-- move_no_pair: integer (nullable = true)
 |-- player: string (nullable = true)
 |-- notation: string (nullable = true)
 |-- move: string (nullable = true)
 |-- from_square: string (nullable = true)
 |-- to_square: string (nullable = true)
 |-- piece: string (nullable = true)
 |-- color: string (nullable = true)
 |-- fen: string (nullable = true)
 |-- is_check: integer (nullable = true)
 |-- is_check_mate: integer (nullable = true)
 |-- is_fifty_moves: integer (nullable = true)
 |-- is_fivefold_repetition: integer (nullable = true)
 |-- is_game_over: integer (nullable = true)
 |-- is_insufficient_material: integer (nullable = true)
 |-- white_count: integer (nullable = true)
 |-- black_count: integer (nullable = true)
 |-- white_pawn_count: integer (nullable = true)
 |-- black_pawn_count: integer (nullable = true)
 |-- white_queen_count: integer (nullable = true)
 |-- black_queen_count: integer (null

In [ ]:
# View data
moves.show(5)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
# schema validation 
eco_codes.printSchema()

root
 |-- eco: string (nullable = true)
 |-- eco_name: string (nullable = true)
 |-- eco_example: string (nullable = true)
 |-- eco_type: string (nullable = true)
 |-- eco_group: string (nullable = true)



In [ ]:
#view data 
eco_codes.show(3)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 3 rows



In [ ]:
# Here We convert string date data to Date format and rename location from site
from pyspark.sql.functions import col 
from pyspark.sql.functions import to_date
game_info = game_info \
            .withColumn("date_played",to_date(col("date_played"),"yyyy.MM.dd"))
game_info =game_info.withColumnRenamed("site","location")
game_info.show(3)


+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event| location|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021-11-26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
# schema validation after process
game_info.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date_played: date (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



# Section 2-- Using PySpark for Data analysis

In [ ]:
# In given data there are different type Event are played.
# But we want to process 'World Championship' Event Only

In [ ]:
# Creating TempView table to filter World Champ series only
wcp2 = game_info \
          .filter((~ game_info.event.contains('KO')) \
                  &(~ game_info.event.contains('k.o')) \
                  & (game_info.tournament_name.contains("WorldChamp")))
wcp2.createOrReplaceTempView("wcp")
wcp = spark.sql("SELECT * FROM wcp")
wcp.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event| location|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021-11-26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
# Checking filter Tournament only contain WCP event only
wcp.groupBy('event').count().show(5)

+--------------------+-----+
|               event|count|
+--------------------+-----+
|World Championshi...|   32|
|World Championshi...|   22|
|World Championshi...|   26|
|World Championshi...|   25|
|World Championshi...|   21|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# List of Winners of Each World champions Trophy.

from pyspark.sql.functions import split
wcp12 =wcp.filter((wcp.winner != 'draw'))
wcp1 = wcp12.withColumn("Player", split(col("winner"), ",").getItem(0))
wce =(wcp1.groupBy(['tournament_name','Player']).max())
wcd = wce.orderBy(wce.tournament_name.desc(),wce['max(game_order)'].desc())
wc =wcd.dropDuplicates(['tournament_name']).orderBy(wcd.tournament_name.desc(),wcd['max(game_order)'].desc())
wc.createOrReplaceTempView('winner')
winner = spark.sql("select player as winner,tournament_name  from winner")
spark.catalog.dropTempView('winner')
df1 =winner
winner.show(10)

+-------+---------------+
| winner|tournament_name|
+-------+---------------+
|Carlsen| WorldChamp2021|
|Carlsen| WorldChamp2016|
|Carlsen| WorldChamp2014|
|Carlsen| WorldChamp2013|
|  Anand| WorldChamp2012|
|  Anand| WorldChamp2010|
|Kramnik| WorldChamp2008|
|Kramnik| WorldChamp2007|
|Kramnik| WorldChamp2006|
|Kramnik| WorldChamp2004|
+-------+---------------+
only showing top 10 rows



In [ ]:
#  List of Players with number of times they have won Tournament in descending order(Max to min).

no_title = wcd.groupBy('Player').count()
no_title = no_title.withColumnRenamed('count','number_of_wins')
no_title = no_title.orderBy(no_title['number_of_wins'].desc())
df2 = no_title
no_title.show()


+----------+--------------+
|    Player|number_of_wins|
+----------+--------------+
| Botvinnik|             8|
|    Lasker|             7|
|    Karpov|             7|
|  Steinitz|             6|
|     Anand|             5|
|   Kramnik|             5|
|  Alekhine|             5|
|  Kasparov|             5|
|   Carlsen|             4|
|   Smyslov|             4|
|   Spassky|             3|
|      Euwe|             3|
| Petrosian|             3|
|Capablanca|             2|
|       Tal|             2|
|Bogoljubow|             2|
|Kortschnoj|             2|
|      Leko|             2|
|   Topalov|             2|
|  Chigorin|             2|
+----------+--------------+
only showing top 20 rows



In [ ]:
# Most and Least Popular eco move in world championship history.

from pyspark.sql.functions import max as sparkMax, min as sparkMin
eco_move = wcp1.groupBy('eco').count()
eco_move = eco_move.withColumnRenamed('count','most')
eco_move = eco_move.join(eco_codes,eco_move.eco == eco_codes.eco ,'inner').select(eco_move.eco,eco_codes.eco_name,eco_move.most)
eco_move.createOrReplaceTempView('Most')
eco_move = spark.sql("""select * from Most
                             where most = (select max(most) from Most) OR most = (select min(most) 
                             from Most) 
                             order by most DESC;""")
df3 = eco_move
eco_move.show( 2 )

+---+--------------------+----+
|eco|            eco_name|most|
+---+--------------------+----+
|C52|        Evans Gambit|  15|
|B34|Sicilian, Acceler...|   1|
+---+--------------------+----+
only showing top 2 rows



In [ ]:
# Find the eco move with most winnings.
eco_winner = game_info.filter(game_info.white == game_info.winner)
eco_winner = (eco_winner.groupBy('eco').count()).withColumnRenamed('count','Use_to_open')
eco_winner = eco_winner.join(eco_codes,eco_winner.eco == eco_codes.eco, 'inner')
eco_winner = eco_winner.orderBy(eco_winner['Use_to_open'].desc()).select(eco_codes.eco,eco_codes.eco_name,eco_winner.Use_to_open)
eco_winner.createOrReplaceTempView('eco_win')
df4 = eco_winner
eco_winner.show()

+---+--------------------+-----------+
|eco|            eco_name|Use_to_open|
+---+--------------------+-----------+
|B90|   Sicilian, Najdorf|         19|
|C11|              French|         17|
|C88|           Ruy Lopez|         16|
|C42|      Petrov Defense|         15|
|D37|Queen's Gambit De...|         14|
|C65|Ruy Lopez, Berlin...|         14|
|C45|         Scotch Game|         13|
|C92|   Ruy Lopez, Closed|         11|
|B12|   Caro-Kann Defense|         11|
|B19|Caro-Kann, Classical|         10|
|C52|        Evans Gambit|         10|
|D17|Queen's Gambit De...|         10|
|C67|           Ruy Lopez|         10|
|D46|Queen's Gambit De...|          9|
|D30|Queen's Gambit De...|          8|
|C18|     French, Winawer|          8|
|D27|Queen's Gambit Ac...|          8|
|E15|      Queen's Indian|          8|
|B51|Sicilian, Canal-S...|          8|
|B48|Sicilian, Taimano...|          7|
+---+--------------------+-----------+
only showing top 20 rows



In [ ]:
# Longest and shortest game ever played in a world championship in terms of move.


move_count = moves.groupby('game_id').count()
move_count = move_count.filter(move_count['count'] >1 ).withColumnRenamed('count','game_lenght')
move_count = move_count.join(wcp1,move_count.game_id ==wcp1.game_id, "inner") \
                    .select(move_count.game_id,wcp1.event,wcp1.tournament_name,move_count.game_lenght)
move_count.createOrReplaceTempView('movels')
movels = spark.sql("""
            SELECT game_id,event, tournament_name,int(game_lenght/2) as number_of_moves 
            FROM movels
            WHERE game_lenght = (SELECT max(game_lenght) FROM movels)
                     OR game_lenght = (SELECT min(game_lenght) FROM movels)
            """)
df5 = movels
movels.show()

+--------------------+--------+---------------+---------------+
|             game_id|   event|tournament_name|number_of_moves|
+--------------------+--------+---------------+---------------+
|646031d8-05e7-40b...|WCh 2012| WorldChamp2012|             16|
|58e83255-93bb-4d5...|WCh 2021| WorldChamp2021|            135|
+--------------------+--------+---------------+---------------+



In [ ]:
# Shortest and Longest Draw game ever Played.

draw1 = game_info.filter(game_info.winner == 'draw')
draw = moves.groupby('game_id').count()
draw = draw.filter(draw['count'] >1 ).withColumnRenamed('count','game_lenght')
draw = draw1.join(draw,draw.game_id ==draw1.game_id, "inner") \
                              .select(draw.game_id,draw1.event,draw1.tournament_name,draw.game_lenght)
draw.createOrReplaceTempView('draw')
draw = spark.sql("""
            SELECT game_id,event, tournament_name,int(game_lenght/2) as number_of_moves 
            FROM draw
            WHERE game_lenght = (SELECT max(game_lenght) FROM draw)
                     OR game_lenght = (SELECT min(game_lenght) FROM draw)
            """)
df6 =draw
draw.show()

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            129|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|              6|
+--------------------+-------------+---------------+---------------+



In [ ]:
# Most and Least rated Player

rating = game_info.filter((game_info.white_elo >0) |(game_info.black_elo >0))
rating.createOrReplaceTempView('rating')
rating = spark.sql("""SELECT  DISTINCT black as  player_name,white_elo as elo from rating
                                where white_elo == (select min(white_elo) from rating) 
                                OR white_elo == (select max(white_elo) from rating)""")
df7 = rating
rating.show()

+------------------+----+
|       player_name| elo|
+------------------+----+
|         Topalov,V|2076|
|Anand, Viswanathan|2870|
+------------------+----+



In [ ]:
# 3rd Last Player with most Loss 

most_loss = game_info.groupBy(game_info.loser).count()
most_loss = most_loss.orderBy(most_loss['count'].desc())
most_loss = most_loss.limit(4)
most_loss = most_loss.withColumnRenamed('count','tot')
most_loss.createOrReplaceTempView('most_loss')
most_loss =spark.sql("""
                        (SELECT loser as player_name from most_loss
                        ORDER BY tot
                        limit 1)
                        """)
df8 =most_loss
most_loss.show()

+---------------+
|    player_name|
+---------------+
|Karpov, Anatoly|
+---------------+



In [ ]:
# How many times players with low rating won matches with their total win Count.

low_rat = game_info.filter(game_info.winner_elo < game_info.loser_elo)
low_rat = (low_rat.groupBy('winner').count()).withColumnRenamed('count','win_count')
low_rat =low_rat.orderBy(low_rat.win_count.desc())
df9 = low_rat
low_rat.show()

+--------------------+---------+
|              winner|win_count|
+--------------------+---------+
|     Karpov, Anatoly|       20|
|      Kasimdzhanov,R|       11|
|           Macieja,B|        9|
|           Kramnik,V|        8|
|  Kortschnoj, Viktor|        8|
|          Grischuk,A|        7|
|        Ponomariov,R|        7|
|Khalifman, Alexander|        6|
|      Kasparov, Gary|        5|
|   Akopian, Vladimir|        5|
|           Sulskis,S|        5|
|      Short, Nigel D|        5|
|         Dominguez,L|        5|
|           Ehlvest,J|        5|
|           Gelfand,B|        4|
|Nisipeanu, Liviu ...|        4|
|         Filippov,Va|        4|
|          Nakamura,H|        4|
| Tkachiev, Vladislav|        4|
|            Volkov,S|        4|
+--------------------+---------+
only showing top 20 rows



In [ ]:
# Move Sequence for Each Player in a Match

seq =moves.orderBy(moves['move_no'].desc())
seq = (seq.groupBy(['game_id','player']).count()).withColumnRenamed('count','move_no')
seq1 =(seq.join(moves, ['move_no','game_id','player'],'inner')\
       .select('game_id','player','move_sequence','move_no_pair'))\
       .withColumnRenamed('move_no_pair','move_count')
seq1 = seq1.orderBy(seq1.game_id.desc())
df10 = seq1
seq1.show()

+--------------------+-------------------+--------------------+----------+
|             game_id|             player|       move_sequence|move_count|
+--------------------+-------------------+--------------------+----------+
|ffdfe902-b93f-4b5...|       Minasian,Art|e4|e6|d4|d5|Nc3|B...|        18|
|ffdfe902-b93f-4b5...|       Villamayor,B|e4|e6|d4|d5|Nc3|B...|        17|
|ffcd8f69-ec31-41a...| Tarrasch, Siegbert|e4|e5|Nf3|Nc6|Bb5...|        22|
|ffb635d0-7e47-49b...|  Ivanchuk, Vassily|c4|e5|Nc3|Nc6|Nf3...|        15|
|ff89b89b-81cb-44c...|         Grischuk,A|d4|Nf6|c4|e6|g3|d...|        30|
|ff77eb24-43ad-4f7...|      Piket, Jeroen|d4|Nf6|c4|g6|Nc3|...|        26|
|ff728052-ff60-4ed...|          Topalov,V|e4|e5|Nf3|Nf6|Nxe...|        16|
|ff728052-ff60-4ed...|           Shirov,A|e4|e5|Nf3|Nf6|Nxe...|        15|
|ff6a555b-8e5d-4a4...|         Vaganian,R|d4|d5|c4|e6|Nc3|B...|        25|
|ff6a555b-8e5d-4a4...|           Bareev,E|d4|d5|c4|e6|Nc3|B...|        26|
|ff40884b-817b-4d6...|   

In [ ]:
# Total Number of games where losing player has more Captured score than Winning player.
capt = moves.join(game_info,'game_id','inner')
capt = capt.filter((moves.player == game_info.loser))
capt = capt.groupBy('game_id','player').max().select('game_id','player','max(captured_score_for_white)','max(captured_score_for_black)')
capt = capt.join(game_info,'game_id','inner').select('game_id','player','max(captured_score_for_white)','max(captured_score_for_black)','white')
capt = capt.filter(((moves['player']==game_info['white']) & (capt['max(captured_score_for_white)'] > capt['max(captured_score_for_black)'])) |((moves['player']==game_info['black']) & (capt['max(captured_score_for_white)'] < capt['max(captured_score_for_black)']) ))
capt.createOrReplaceTempView('capt')
capt = spark.sql('select COUNT(*) as total_number_of_games from capt')
df11 = capt
capt.show()

+---------------------+
|total_number_of_games|
+---------------------+
|                  209|
+---------------------+



In [ ]:
# List All Perfect Tournament with Winner Name.
perf_win = game_info.filter((game_info.winner != 'draw'))
perf1 = perf_win.groupBy(['tournament_name','winner']).count().alias('wins').select(['tournament_name','winner'])
perf2 = perf_win.groupBy(['tournament_name','loser']).count().alias('loses').\
                          withColumnRenamed('loser','winner').select(['tournament_name','winner'])
perf = perf1.join(perf2,[perf1.tournament_name==perf2.tournament_name,perf1.winner == perf2.winner],'fullouter' )\
                                            .filter(perf2.winner.isNull()).\
                                            select(perf1['winner'],perf1['tournament_name'])
perf = perf.withColumnRenamed('winner','winner_name')
df12 = perf
perf.show()

+--------------------+---------------+
|         winner_name|tournament_name|
+--------------------+---------------+
|Capablanca, Jose ...| WorldChamp1921|
|          Ciuksyte,D|  FideChamp2002|
|            Karlov,A|  FideChamp2004|
|           Carlsen,M| WorldChamp2021|
|       Zielinska,Mar|  FideChamp2002|
|Stepovaia Dianche...|  FideChamp2002|
|           Topalov,V|  FideChamp2005|
|     Lasker, Emanuel| WorldChamp1907|
|     Carlsen, Magnus| WorldChamp2013|
|            Foisor,C|  FideChamp2002|
|             Anand,V| WorldChamp2007|
|             Anand,V|  FideChamp2000|
|     Lasker, Emanuel|WorldChamp1910b|
|           Kramnik,V| WorldChamp2000|
|          Kiseleva,N|  FideChamp2002|
|  Anand, Viswanathan| WorldChamp2012|
+--------------------+---------------+



In [ ]:
# Player with highest winning ratio.
movess = moves.join(wcp,'game_id','inner')
tot_played = movess.groupBy('game_id','player').count().withColumnRenamed('count','played').select('player','played')
tot_played = tot_played.groupBy('player').count().withColumnRenamed('count','played').select('player','played')
tot_win = wcp.groupBy('winner').count().withColumnRenamed('count','wins').select('winner','wins')
win_ratio = tot_played.join(tot_win,tot_played.player == tot_win.winner,'inner')
win_ratio.createOrReplaceTempView('ratio')
ratio = spark.sql("""
                      SELECT player as player_name,played,wins,float(wins*100/played) as win_ratio from ratio
                      ORDER BY win_ratio DESC
                      limit 1
                    """)
ratio =ratio.select('player_name')
df13 = ratio
ratio.show()

+---------------+
|    player_name|
+---------------+
|Lasker, Emanuel|
+---------------+



In [ ]:
# Player who had given checkmate with Pawn
movess = moves.join(wcp,'game_id','inner')
cp = movess.filter((movess.is_check_mate >=1)\
                  &(((movess.color == 'Black') & (movess.white_pawn_count >=1))\
                    |((movess.color == 'White')& (movess.black_pawn_count >=1)))).select('player')
df14 = cp
cp.show()

+-------------------+
|             player|
+-------------------+
|Alekhine, Alexander|
+-------------------+



In [ ]:
#List games where player has won game without queen.

movess = moves.join(game_info,'game_id','inner')
queenwo = movess.filter((movess.is_check_mate >=1)\
                  &(((movess.color == 'White') & (movess.white_queen_count ==0))\
                    |((movess.color == 'Black')& (movess.black_queen_count ==0))))\
                    .withColumnRenamed('player','player_name').select('game_id','event','player_name')
df15 =queenwo
queenwo.show()


+--------------------+-------------+--------------+
|             game_id|        event|   player_name|
+--------------------+-------------+--------------+
|04c81fe4-efd8-456...|  FIDE WCh KO|    Nakamura,H|
|ac45a5ed-172d-4ae...|FIDE-Wch k.o.|Andersson, Ulf|
+--------------------+-------------+--------------+



# Section 3

In [ ]:
# Function will accept list of spark dataframe as argument.

In [ ]:
def csv_file(fil):
  for i in range(0,15):
      (globals()[fil[i]].toPandas())\
          .to_csv("/content/drive/MyDrive/DE_SOLUTION_ARAVIND_PS/results/{0}.csv".format(fil[i]))

In [ ]:
# Mount your Google Drive if not mounted onto your colab notebook.
try:
  from google.colab import drive
  drive.mount('/content/',force_remount =True)
except ValueError:
  print("Already Created")
else:
  print("Sucessfully Created")


Already Created


In [ ]:
#  Create a directory in your google drive if not created with 
#               name: DE_SOLUTION_FirstName_LastName/results/

import os.path
from os import path
import pandas as pd

if path.exists('/content/drive/MyDrive/DE_SOLUTION_ARAVIND_PS/results/') == False:
  os.mkdir('/content/drive/')
  os.mkdir('/content/drive/MyDrive/')
  os.mkdir('/content/drive/MyDrive/DE_SOLUTION_ARAVIND_PS/')
  os.mkdir('/content/drive/MyDrive/DE_SOLUTION_ARAVIND_PS/results/')
  print(" Directory Created Sucessfully")
else:
  print("Already folder Created")



Already folder Created


In [ ]:
fil = []
for i in range(1,16):
  fil.append('df{0}'.format(i))

In [ ]:
# Convert spark dataframe to pandas dataframe
# Convert the pandas dataframe to csv to '/content/gdrive/MyDrive/DE_SOLUTION_ARAVIND_PS/results/'
csv_file(fil)
print('All results files are saved as df& question Number')

All results files are saved as df& question Number
